In [9]:
import yfinance as yf
import plotly.graph_objs as go
import pandas as pd
from datetime import datetime, timedelta
from sqlalchemy import create_engine

username = 'postgres'
password = 'postgres'
host = 'localhost'
port = '5432'
database = 'finance'
str_conn = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}'

In [3]:
end_date = datetime.now()
start_date = end_date - timedelta(days=365*2)
stock_symbol = 'GOOG'

In [8]:
stock_data = yf.download(stock_symbol, start=start_date, end=end_date)
stock_data['symbol'] = stock_symbol
stock_data = stock_data.reset_index()
stock_data.columns = stock_data.columns.str.lower()
stock_data = stock_data.drop(['adj close'], axis=1)

[*********************100%***********************]  1 of 1 completed


In [53]:
ticker = yf.Ticker(stock_symbol)
list_fields = ['symbol', 'country', 'industry', 'sector', 'fullTimeEmployees', 'ebitda', 'totalDebt', 'totalRevenue', 'debtToEquity', 'grossMargins', 'ebitdaMargins']
ticker = pd.DataFrame.from_dict({k:v for k,v in ticker.info.items() if k in list_fields}, orient='index').T
ticker.columns = ticker.columns.str.lower()

In [58]:
ticker.to_sql('stocks', con=str_conn, if_exists='append', index=False)
stock_data.to_sql('stock_prices', con=str_conn, if_exists='append', index=False)

In [22]:

# Datos de conexión
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = '5432'
database = 'finance'
str_conn = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}'
# Crear el motor de conexión (Engine)
engine = create_engine(str_conn)

# Verificar la conexión
try:
    connection = engine.connect()
    print("Conexión exitosa!")
    
    # Ejecutar una consulta de prueba
    result = connection.execute("SELECT * from public.stock_prices;")
    for row in result:
        print(row)

    
except Exception as error:
    print(f"Error al conectarse a la base de datos: {error}")
    
finally:
    connection.close()
    print("Conexión cerrada.")

Conexión exitosa!
Error al conectarse a la base de datos: Not an executable object: 'SELECT * from public.stock_prices;'
Conexión cerrada.


In [33]:
from sqlalchemy.sql import text

connection = engine.connect()
result = connection.execute(text('SELECT * from stocks'))
connection.close()